In [2]:
def file_len(fname):
    with open(fname, encoding='utf-8') as f:
        for i, l in enumerate(f):
            pass
    return i + 1
print(file_len('polish_corpora.txt'))

23011601


## Gather words

In [3]:
def gather_words_from_file(file_name, lines_to_read):
    words = {}
    with open(file_name, encoding='utf-8') as f:
        for _ in range(lines_to_read):
            words_in_line = next(f).rstrip('\n').split(' ')
            for word in words_in_line:
                words[word.lower()] = True
    return words

In [4]:
words = gather_words_from_file(file_name='polish_corpora.txt', lines_to_read=500000)

## Exercise 1

### a)

In [5]:
def get_score(tokenization_attempt, proper_tokenization):
    score = 0
    for token in tokenization_attempt:
        if token in proper_tokenization:
            score +=1
    return score / len(proper_tokenization)

In [6]:
get_score(['t1', 't2t3', 't3'], ['t1', 't2', 't3', 't3'])

0.5

### b)

In [7]:
def max_match(text):
    tokens = []
    current_token = ''
    for letter in text:
        extended_token = current_token + letter
        if extended_token in words:
            current_token = extended_token
        else:
            tokens.append(current_token)
            current_token = letter
    if current_token != '':
        tokens.append(current_token)
    return tokens

In [8]:
def tokenize_and_compare(line):
    proper_tokenization = line.lower().split(' ')
    processed_line = "".join(line.split()).lower()
    attempted_tokenization = max_match(processed_line)
    score = get_score(attempted_tokenization, proper_tokenization)
    print(f'expected: {proper_tokenization}')
    print(f'got: {attempted_tokenization}')
    print(f'score: {score}')
    print()

In [9]:
with open('polish_corpora.txt', encoding='utf8') as f:
    for _ in range(5):
        line = next(f).rstrip('\n')
        tokenize_and_compare(line)

expected: ['system', 'euroraty', 'chcesz', 'kupować', 'więcej', 'niż', 'gdzie', 'indziej', '?']
got: ['syst', 'eme', 'uro', 'raty', 'ch', 'ces', 'zk', 'upo', 'wać', 'więc', 'ej', 'niż', 'gd', 'zie', 'ind', 'zie', 'j', '?']
score: 0.2222222222222222

expected: ['parlament', 'zdecydował', 'jednak', 'inaczej', 'i', 'przyjął', 'w', 'ustawie', 'z', 'dnia', '28.06.1996', 'r.', 'jednoinstancyjne', 'postępowanie', 'orzeczniczo-lekarskie', '.']
got: ['par', 'lam', 'en', 'tz', 'dec', 'y', 'dowa', 'ł', 'jednak', 'ina', 'czej', 'ip', 'rzy', 'jął', 'wu', 'stawie', 'zd', 'nia', '28.', '06.', '1996r', '.', 'jedno', 'insta', 'nc', 'yj', 'nep', 'ost', 'ę', 'pow', 'anie', 'orz', 'ec', 'zn', 'ic', 'zo', '-', 'lek', 'ars', 'kie', '.']
score: 0.1875

expected: ['po', 'kampanii', 'wrześniowej', '1.dlek', 'raportowała', '77', 'czołgów', 'l.t.m.35', 'utraconych', '(', 'wraz', 'z', 'wozami', 'dowodzenia', ')', 'i', '52', 'uszkodzone', 'lub', 'zepsute', ',', 'lecz', 'ostatecznie', 'po', 'naprawach', 'straty', '

#### The file is kind of scuffed ngl

In [10]:
with open('polish_corpora.txt', encoding='utf-8') as f:
        for _ in range(500000):
            line = next(f).rstrip('\n')
            words_in_line = line.split(' ')
            if 'zk' in words_in_line:
                print(line)

Nie ch ju ż naw et nie będ zie str a żn ic zk ą , a le n ie ch w o góle b ęd zie .
Moze nie trzeba by bylo interweniowac a miejsce w zk przydaloby sie komus innemu .
A to , czy jakis pies kryl warunkowo czy nie , czy zk jest fajny czy be ...


## Exercise 2

In [73]:
from random import choice


class SentenceGenerator:
    def __init__(self, successors):
        self.successors = successors

    def random_word(self):
        return choice(list(self.successors.keys()))

    def random_successor_of(self, word):
        successors = self.successors[word]
        return choice(successors)[1].split(' ')

    def has_successors(self, word):
        return word in self.successors and len(self.successors[word]) > 0

    def is_end_of_sentence(self, word):
        return word in ['<EOS>', '.', '?', '!', '?!', '!?']

    def generate_sentence(self):
        word = self.random_word()
        while not self.has_successors(word):
            word = self.random_word()

        sentence = [word]
        while self.has_successors(word):
            successor = self.random_successor_of(word)
            word = successor[-1]
            sentence += successor

        return ' '.join(sentence) + ('.' if not self.is_end_of_sentence(sentence[-1]) else '')
    
    def generate_text(self, sentences=1):
        return '\n'.join([self.generate_sentence() for _ in range(sentences)])

In [12]:
def add_successor(successors, n, word, successor):
    successor = (n, successor)
    if word in successors:
        successors[word].append(successor)
    else:
        successors[word] = [successor]

### Bigrams

In [13]:
def gather_successors_from_bigrams(file_name, K=1):
    successors = {}
    with open(file_name, encoding='utf-8') as f:
        line = f.readline()
        while line:
            n, word1, word2 = line.rstrip('\n').split(' ')
            if (int(n) >= K):
                add_successor(successors, int(n), word1, word2)
            line = f.readline()
    return successors

In [14]:
successors = gather_successors_from_bigrams('poleval_2grams.txt', 10)

In [64]:
sg = SentenceGenerator(successors)
sg.generate_text(sentences=5)

'smuty . alfreda dreyfusa , wałbrzych w priorytecie viii 1943 walczył zarówno kierownictwo centralnego : zwykłej - kronika parafii dekanatu jeleśnia , wynegocjowane okresy jej placówki opiekuńczo wychowawcza albo rok 1928 ( płomień sosnowiec to natychmiastowe odwołanie sędziego orzekającego , zegrzu i regenerację tkanek jest uważane jako komercyjny ( fsc . większość zgromadzonych informacji społecznej dotyczącej potrzeby hodowli tych ram prawnych zawierają dzieła jako zasadniczo w północnoeuropejskiej był energicznym i 2318.\ndrzewcu sztandaru przez polkę i alexa fergusona . prawo nakazywało.\ncjo nar . łukasza ) ktory jest proces konsolidacji inspekcji farmaceutycznej w rozrywce w indywidualnej umowy dotychczas istniejąca obecnie ruiny dwóch użytkowników do lęgów i myśliwskie i ...... i reprodukcje . 8d.\nzawitać na śledztwie ? <EOS>\npiernikami , odbicie również zastosowania regulacji o konserwację obwałowań oraz matki boskiej piekarskiej , wyłączającego obowiązek uzyskać takich soju

### Trigrams

In [16]:
def gather_successors_from_trigrams(file_name, K=1):
    successors = {}
    with open(file_name, encoding='utf-8') as f:
        line = f.readline()
        while line:
            line = line.rstrip('\n').split(' ')
            if (len(line) != 4): break
                
            n, word1, word2, word3 = line
            if (int(n) >= K):
                add_successor(successors, int(n), word1, f'{word2} {word3}')
            line = f.readline()
    return successors

In [17]:
successors3 = gather_successors_from_trigrams('poleval_3grams.txt', 2)

In [18]:
sg3 = SentenceGenerator(successors3)
sg3.generate_text(sentences=5)

'naukowo-techniczną i gospodarczą. inne są nie przysługuje przez to pokolenie. przed dwudziestym rokiem. latem na igelicie. miejsce rozgrywania wyścigu.'

## Exercise 3

In [71]:
import numpy as np

class NonUniformSentenceGenerator(SentenceGenerator):
    def random_successor_of(self, word):
        successors = self.successors[word]
        total_occurences = sum(list(map(lambda s: s[0], successors)))
        probabilites = list(map(lambda s: s[0] / total_occurences, successors))
        successors = list(map(lambda s: s[1], successors))

        return [np.random.choice(successors, 1, probabilites)[0]]

In [76]:
nusg2 = NonUniformSentenceGenerator(successors)
print(nusg2.generate_text(sentences=5))

boecjusz , wchłonięta przez rzeźbiarzy i skorzystają na żyłę , powiadamia się zatnie.
sese seko . kapitał polski klub jednak zasady pluralizmu związkowego , odcinkiem specjalnym koncercie z działów kadr jest upoważnienie nie żarty z dachem trójspadowym dachem - warunkiem zatwierdzenia tej dacie premiery to otrzymaliśmy dopiero druga kwestia najbliższych wyborów prezydenckich do fizjologicznego porodu rodzinnego wraz . bonifratrów . urz . władysław kalkus.
ge- netics.
ustawiczne , 586 p.n.e. prawdopodobnie przez przewodniczących właściwych parametrach zbliżonych rozmiarach niż pokazują przynajmniej kilkuset cywilów na dyrektorskim stanowisku pracownika fizycznego o niecałe 900 mm to udostępnianie zawartości zanieczyszczeń odprowadzanych w oficynie , zbycie nastąpiło zaniechanie przy oglądaniu telewizji publicznych wynika ich wychowanie muzyczne mp3 ... macie czas co były ułatwione jest rzędem temu dostał posadę szefa łączności 1 wskazuje do niedotlenienia . 52 cm kolor można obecnie spr

In [74]:
nusg3 = NonUniformSentenceGenerator(successors3)
print(nusg3.generate_text(sentences=5))

indianami północnego zachodu.
czwarta w tej.
oceniana była jako.
wielu wypadkach gminy.
sieci odpowiadający zapotrzebowaniu.
